In [1]:
import re
import json
import argparse
import openai
import configparser
import os


In [38]:

def get_config_apikey(cfg_file):
    config = configparser.ConfigParser()
    config.read(cfg_file)
    api_key = config.get('Config', 'openai-apikey', fallback=None)
    return api_key

def get_config_code(cfg_file):
    config = configparser.ConfigParser()
    config.read(cfg_file)
    code = config.get('Config', 'code_language', fallback=None)
    return code

def get_config_prompt(cfg_file):
    config = configparser.ConfigParser()
    config.read(cfg_file)
    prompt = config.get('Config', 'codeMaster_prompt', fallback=None)
    return prompt

api_key = get_config_apikey('./.cfg')
master_prompt = get_config_prompt('./.cfg')
openai.api_key = api_key

def find_segments_c(text):
    segments = []
    stack = []
    start = 0
    for i, char in enumerate(text):
        if char == '{':
            if i > start:
                # save the code segment before this {
                segments.append(text[start:i].strip())
            stack.append(i)
            start = i
        elif char == '}':
            stack.pop()
            if not stack:
                # save this function
                segments.append(text[start:i+1].strip())
                start = i+1
    # save the last code segment
    if start < len(text):
        segments.append(text[start:].strip())
    return segments

def print_json(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    for segment in data:
        print(segment)
        print()



    
def askGpt(*args):
    prompt = ' '.join(args)
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=master_prompt+prompt,
        temperature=0,
        max_tokens=64,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        stop=["\"\"\""]
    )
    generated_text = response['choices'][0]['text'].strip()
    print(generated_text)
    return generated_text





In [39]:
def process_file(input_file):
    output_filename = 'codingMaster_' + input_file

    with open(input_file, 'r') as infile, open(output_filename, 'w') as outfile:
        for line in infile:
            stripped_line = line.strip()
            if stripped_line == "":  # 如果是空白行，則跳過此次迴圈
                outfile.write('\n')  # 寫入空行
                continue
            if '//' in stripped_line or '{' in stripped_line or '}' in stripped_line:  # 如果該行已經包含 "//"、"{" 或 "}"
                outfile.write(stripped_line + '\n')  # 寫入原始行
                continue
            print(stripped_line)
            processed_line = askGpt(stripped_line)  # 對每一行調用 askGpt 函數
            #processed_line = "註解"
            outfile.write(stripped_line + ' //' + processed_line + '\n')  # 將處理後的結果追加到該行並寫入新檔案

    print(f'Processed file has been saved to: {output_filename}')



In [26]:
 askGpt("這是什麼，請以最短的字數幫我做標注","#include <stdint.h>")

#C語言標頭檔


'#C語言標頭檔'

In [40]:
input_file = 'arduino.ino'  
process_file(input_file)


#include <stdint.h>
C語言標頭檔
#include <stdio.h>
C語言標頭檔
#include <stdlib.h>
C語言標頭檔
#include <avr/io.h>
AVR C語言函式庫
#include <avr/interrupt.h>
AVR中斷函式庫
#include <util/delay.h>
延遲函數庫
#define SET_BIT(reg, pin)		    (reg) |= (1 << (pin))
設定位元（reg, pin）
#define CLEAR_BIT(reg, pin)		  (reg) &= ~(1 << (pin))

void setup(void);
設定函數
void process(void);
函數定義
TCCR1A = 0;
TCCR1A：定時器/計數器控制寄存器1A，值為0。
CLEAR_BIT(TCCR1B,WGM12);
設定TCCR1B的WGM12位為0
CLEAR_BIT(TCCR1B,WGM13);
設定TCCR1B的WGM13位為0
SET_BIT(TCCR1B,CS12);
設定TCCR1B的CS12位為1
CLEAR_BIT(TCCR1B,CS11);
清除TCCR1B的CS11位
SET_BIT(TCCR1B,CS10);
設定TCCR1B的CS10位為1
SET_BIT(TIMSK1, TOIE1);
定義TIMSK1中TOIE1位為1
SET_BIT(DDRB, 5);
設定PB5為輸出
sei();
未知物。
volatile int overflow_counter = 0;
變數: overflow_counter, 數值: 0, 資料型態: volatile int
overflow_counter ++;
計數器增加1。
SET_BIT(PORTB, 5);
設定PORTB的第5位為1
setup();
設定程序。
process();
函數
_delay_ms(1000);
時間延遲。
Processed file has been saved to: codingMaster_arduino.ino
